In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import student
from main import load_data

import numpy as np
import matplotlib.pyplot as plt
import scipy
import development
from main import load_data
from skimage import io, filters, feature, img_as_float32
from skimage.transform import rescale
from skimage.color import rgb2gray
from helpers import cheat_interest_points
np.set_printoptions(precision=8, suppress=True)

# Feature matching

For feature matching we use the **nearest neighbor distance ratio (NDDR)** metric to measure the distance between the feature vectors of two images. D1 and D2 are the nearest and second nearest neighbor distaces. 

$NDDR = \frac{d_1}{d_2} = \frac{||D_A - D_B||}{||D_A - D_C||}$

In [ ]:
im1_features = np.array([[1], [200], [100]])
im2_features = np.array([[1.1], [1.2], [202]])
matches, confidences = development.match_features(im1_features, im2_features)

In [ ]:
matches, confidences

# Feature vector generation

In [ ]:
x = np.array([7, 7])
y = np.array([7, 7])
feature_width = 16
imgsize = 18
image = np.arange(0,imgsize*imgsize).reshape(imgsize,imgsize)
imgsize

In [ ]:
feat = development.get_features(image,x,y,feature_width)

In [ ]:
n_bins = 8
gradients = np.array(np.gradient(image))
magnitudes = np.linalg.norm(gradients, axis=0)
angles = np.angle(np.arctan2(gradients[0], gradients[1]), deg=True)
bins = np.digitize(angles,np.arange(0,360,360 // n_bins))

In [ ]:
feature_vector = np.zeros(n_bins)
for i in range(0,n_bins):
    mask = np.array(bins == i).flatten()
    feature_vector[i] = np.sum(magnitudes.flatten()[mask])

In [ ]:
feature_vector

# Interest point detection

In [ ]:
from skimage.measure import regionprops
from skimage.feature import peak_local_max

image1, image2, eval_file = load_data("e_gaudi")
image1 = rgb2gray(image1) * 255
image2 = rgb2gray(image2) * 255
scale_factor = .5
image1 = np.float32(rescale(image1, scale_factor, multichannel=False))
image2 = np.float32(rescale(image2, scale_factor, multichannel=False))
plt.imshow(image1, cmap="gray")

In [ ]:
image = image1
sigma = 3
# TODO consider doing this in one step.
image_blurred = filters.gaussian(image, sigma)
Iy, Ix = np.gradient(image_blurred)
Ixx = filters.gaussian(Ix * Ix, sigma)
Iyy = filters.gaussian(Iy * Iy, sigma)
Ixy = filters.gaussian(Ix * Iy, sigma)

In [ ]:
alpha = 0.04
R = Ixx * Iyy - Ixy**2 - alpha * (Ixx + Iyy)**2
R_norm = (R-np.min(R))/(np.max(R)-np.min(R))
plt.imshow(R_norm, cmap="gray")

In [ ]:
from skimage.feature import peak_local_max
corners = R_norm
threshold = np.mean(R_norm)
#mask = [corners<threshold]
#corners[mask] = 0
plt.imshow(corners, cmap="gray")
keypoints = peak_local_max(corners, threshold_rel=0.3)
keypoints.shape